In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D,Dropout, Reshape, LSTM, BatchNormalization,TimeDistributed
from keras.layers import Conv2D
from keras.optimizers import Adam #, RMSprop, SGD
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from time import time

import pickle
import random
import numpy as np
import sys
import h5py
import glob
import os

Using TensorFlow backend.


In [2]:
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 60
rows, cols = 431, 128   
nb_epoch = 200
pool_size = (5,5)                  # size of pooling area for max pooling
prob_drop_conv = 0.3                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.3              # drop probability for dropout @ fc layer

In [3]:
#might need to rewrite these



def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('trainfiles: ', str(len(trainlist)))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return trainlist,validationlist,testlist

labelRef={'airport-barcelona':0,'shopping_mall-barcelona':1,'metro_station-barcelona':2,'street_pedestrian-barcelona':3,'public_square-barcelona':4, 
          'street_traffic-barcelona':5, 'tram-barcelona':6,'bus-barcelona':7,'metro-barcelona':8,'park-barcelona':9,
          'airport-helsinki':10,'shopping_mall-helsinki':11,'metro_station-helsinki':12,'street_pedestrian-helsinki':13,'public_square-helsinki':14, 
          'street_traffic-helsinki':15,'tram-helsinki':16,'bus-helsinki':17,'metro-helsinki':18,'park-helsinki':19,
          'airport-london':20,'shopping_mall-london':21,'metro_station-london':22,'street_pedestrian-london':23,'public_square-london':24,
          'street_traffic-london':25,'tram-london':26,'bus-london':27,'metro-london':28,'park-london':29,
          'airport-paris':30,'shopping_mall-paris':31,'metro_station-paris':32,'street_pedestrian-paris':33,'public_square-paris':34,
          'street_traffic-paris':35,'tram-paris':36,'bus-paris':37,'metro-paris':38,'park-paris':39,
          'airport-stockholm':40, 'shopping_mall-stockholm':41,'metro_station-stockholm':42,'street_pedestrian-stockholm':43,'public_square-stockholm':44,
          'street_traffic-stockholm':45,'tram-stockholm':46,'bus-stockholm':47,'metro-stockholm':48,'park-stockholm':49,
          'airport-vienna':50,'shopping_mall-vienna':51,'metro_station-vienna':52,'street_pedestrian-vienna':53,'public_square-vienna':54,
          'street_traffic-vienna':55,'tram-vienna':56,'bus-vienna':57,'metro-vienna':58,'park-vienna':59}

def getData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=np.zeros(((len(flist)),nb_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        location='-'.join(tfile.split('\\')[1].split('-')[0:2])
        Y_[i,labelRef[location]]=1
    return X_, Y_
#trainlist,validationlist,testlist=fileLists()
#X_train, Y_train = getData(trainlist)
#X_val, Y_val = getData(validationlist)
#X_test, Y_test = getData(testlist)

print(' data sorted')


 data sorted


In [4]:
###### Convolutional model
def compileCRNN(cols,rows,nb_classes=1):
    model = Sequential()
    # conv1 layer
    model.add(Conv2D(32, (7, 7), padding='same', activation='relu', input_shape=(cols,rows,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv2 layer
    model.add(Conv2D(64, (7, 7), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4,7), strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv3 layer
    model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(prob_drop_conv))

    model.add(Flatten())
    # fc1 layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(prob_drop_hidden))
    model.add(BatchNormalization())
   
    # fc2 layer
    model.add(Dense(nb_classes, activation='softmax'))
    opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = compileCRNN(cols,rows,nb_classes=nb_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 216, 64)       256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 108, 64)       0         
__________

In [6]:
# Train
#fold=1
def train_network(model, model_name, X_train, Y_train, X_val, Y_val, nb_epoch, validationsplit_size, batchsize, early_stoping_patience, output_folder):
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_folder,model_name + '.hdf5'),save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stoping_patience)
    tensorboard = TensorBoard(log_dir="logs\\{}".format(time()))
    Callbacks=[checkpointer,  tensorboard] #early_stopping,
  #  print(samples)
    steps=int(samples/batchsize)
    validationsteps=int(validationsplit_size/batchsize)
    history = model.fit(X_train, Y_train, epochs=nb_epoch, callbacks=Callbacks, batch_size=batch_size, validation_data=(X_val, Y_val), shuffle=True, verbose=1)
    return history,model

def buildModel(savemodelfilename, samples,model,X_train, Y_train,X_val, Y_val):
    valSplit_size = int(samples/4)
    early_stoping_patience=5
    history,model = train_network(model, 'models\\best_model60_simple', X_train, Y_train, X_val, Y_val, nb_epoch, valSplit_size, batch_size, early_stoping_patience,'.')
    model.save_weights(savemodelfilename)
    return model, history


In [7]:
from datetime import datetime
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 60
rows, cols = 431, 128  


trainlist,validationlist,testlist=fileLists()
print('lists sorted')
X_tr, Y_train = getData(trainlist)
X_v, Y_val = getData(validationlist)
X_te, Y_test = getData(testlist)
print('data obtained')
X_train=np.expand_dims(X_tr,axis=3)
X_val=np.expand_dims(X_v,axis=3)
X_test=np.expand_dims(X_te,axis=3)
print(X_train.shape, X_val.shape, X_test.shape)
print('data sorted')
#model = compileCRNN(cols,rows,nb_classes=nb_classes)
model=load_model('models\\best_model60_simple.hdf5')
print('model compiled')
savemodelfilename='models\\final_model60_simple.testsave'
samples=(X_train.shape[0])
print(datetime.now())
models,histories= buildModel(savemodelfilename, samples,model,X_train,Y_train,X_val, Y_val)
print(datetime.now())
print('model saved')

trainfiles:  6122
validationfiles:  2518
testfiles:  2518
lists sorted
data obtained
(6122, 128, 431, 1) (2518, 128, 431, 1) (2518, 128, 431, 1)
data sorted
model compiled
2019-04-08 13:36:38.212036
Train on 6122 samples, validate on 2518 samples
Epoch 1/200
6122/6122 [==============================] - 26s 4ms/step - loss: 1.9333 - acc: 0.4458 - val_loss: 3.2768 - val_acc: 0.1811
Epoch 2/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.8547 - acc: 0.4618 - val_loss: 3.4236 - val_acc: 0.1493
Epoch 3/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.9245 - acc: 0.4392 - val_loss: 2.8287 - val_acc: 0.2216
Epoch 4/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.8581 - acc: 0.4711 - val_loss: 2.7925 - val_acc: 0.2153
Epoch 5/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.9011 - acc: 0.4378 - val_loss: 2.6734 - val_acc: 0.2339
Epoch 6/200
6122/6122 [==============================] - 23s 4ms/step - l

6122/6122 [==============================] - 23s 4ms/step - loss: 1.3218 - acc: 0.6023 - val_loss: 2.7538 - val_acc: 0.2577
Epoch 60/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.3348 - acc: 0.6023 - val_loss: 2.8598 - val_acc: 0.2502
Epoch 61/200
6122/6122 [==============================] - 24s 4ms/step - loss: 1.3405 - acc: 0.5995 - val_loss: 2.9947 - val_acc: 0.2522
Epoch 62/200
6122/6122 [==============================] - 24s 4ms/step - loss: 1.5494 - acc: 0.5356 - val_loss: 3.8369 - val_acc: 0.1751
Epoch 63/200
6122/6122 [==============================] - 24s 4ms/step - loss: 1.3944 - acc: 0.5864 - val_loss: 3.1190 - val_acc: 0.2518
Epoch 64/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.3342 - acc: 0.5980 - val_loss: 2.8866 - val_acc: 0.2681
Epoch 65/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.2919 - acc: 0.6112 - val_loss: 2.6741 - val_acc: 0.2867
Epoch 66/200
6122/6122 [==============================

Epoch 119/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.2209 - acc: 0.6325 - val_loss: 2.4109 - val_acc: 0.3427
Epoch 120/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.0778 - acc: 0.6663 - val_loss: 2.5277 - val_acc: 0.3503
Epoch 121/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.9887 - acc: 0.7043 - val_loss: 2.5997 - val_acc: 0.3407
Epoch 122/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.0972 - acc: 0.6686 - val_loss: 2.4461 - val_acc: 0.3304
Epoch 123/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.9979 - acc: 0.6947 - val_loss: 3.1129 - val_acc: 0.3217
Epoch 124/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.9611 - acc: 0.7099 - val_loss: 6.4625 - val_acc: 0.1692
Epoch 125/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.9880 - acc: 0.6970 - val_loss: 2.3922 - val_acc: 0.3678
Epoch 126/200
6122/6122 [=========

6122/6122 [==============================] - 23s 4ms/step - loss: 0.6205 - acc: 0.8071 - val_loss: 2.5201 - val_acc: 0.3848
Epoch 179/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.6973 - acc: 0.7859 - val_loss: 3.3323 - val_acc: 0.3205
Epoch 180/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.8054 - acc: 0.7501 - val_loss: 2.8203 - val_acc: 0.3018
Epoch 181/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.7873 - acc: 0.7587 - val_loss: 3.4948 - val_acc: 0.2041
Epoch 182/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.9390 - acc: 0.7181 - val_loss: 3.3456 - val_acc: 0.2685
Epoch 183/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.8735 - acc: 0.7342 - val_loss: 2.8527 - val_acc: 0.3495
Epoch 184/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.7323 - acc: 0.7816 - val_loss: 2.6493 - val_acc: 0.3523
Epoch 185/200
6122/6122 [=======================